<a href="https://colab.research.google.com/github/anasnafis77/Deteksi-Glaukoma/blob/main/Glaucoma_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
path = '/content/gdrive/My Drive/Drive Tugas Akhir/Kode/'

In [ ]:
import pandas as pd

df = pd.read_json(path+'dataframe.json')

In [ ]:
def CDR_calc(OD_mask, OC_mask):
  c_OD = cv2.findContours(OD_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  c_OD = cv2.approxPolyDP(c_OD[0], 3, True)
  c_OC = cv2.findContours(OC_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  c_OC = cv2.approxPolyDP(c_OC[0], 3, True)
  ver_OD, hor_OD = cv2.boundingRect(c_OD)
  ver_OC, hor_OC = cv2.boundingRect(c_OC)

  VCDR = ver_OC/ ver_OD # vertical CDR
  HCDR = hor_OC/ ver_OD # Horizontal CDR
  ACDR = np.sum(OC_mask == 255)/ np.sum(OD_mask == 255) # Area CDR

  return VCDR, HCDR, ACDR

def ISNT_calc(OD_mask, OC_mask):
  NRR = OD_mask - OC_mask # Neuro Retinal Rim
  